# **Laboratorio 01**

- **a.** Crea un programa para generar un archivo de tipo texto plano o CSV con más de 1 millón de registros (cualquier lenguaje de programación y sistema operativo). La estructura del archivo debe ser:
  - **(AP, Nom, edad, sexo, peso, talla, color, prov, EscuelaProfesional, fecha_ingreso_U).**

- **b.** Crear un programa pipeline que permita:
  - **i.** La cantidad de varones y mujeres.
  - **ii.** Contar cuántos alumnos hay por cada Escuela Profesional.
  - **iii.** El promedio de edad.
  - **iv.** Cuántas personas ingresaron.

## **1. Crear data**

Crear data de alumnos con 1 000 0000 registros en archivo csv según formato especificado.

In [25]:
import random
import string
from datetime import datetime, timedelta

#Genera string
def getrandomstring(length):
  letters = string.ascii_uppercase
  result_str = ''.join(random.choice(letters) for i in range(length))
  return result_str

#Genera int
def generate_random_int(start, end):
  return random.randint(start, end)

#Genera float
def generate_random_float(start, end):
  return round(random.uniform(start, end), 2)

#Genera fecha
def generate_random_date(start_year, end_year):
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    delta_days = (end_date - start_date).days
    random_days = random.randint(0, delta_days)
    random_date = start_date + timedelta(days=random_days)
    return random_date.strftime("%Y-%m-%d")

#Genera nombre
def generate_name(number, lenght):
  for i in range(number):
    return (getrandomstring(lenght))

#Genera provincia
def generate_province(number):
  if (number == 1):
    return "Abancay"
  elif (number == 2):
    return "Andahuaylas"
  elif (number == 3):
    return "Antabamba"
  elif (number == 4):
    return "Aymaraes"
  elif (number == 5):
    return "Grau"
  elif (number == 6):
    return "Cotabambas"
  elif (number == 7):
    return "Chincheros"
  else:
    return "sin provincia"

#Genera color
def generate_color(number):
  if (number == 1):
    return "Negro"
  elif (number == 2):
    return "Blanco"
  elif (number == 3):
    return "Azul"
  elif (number == 4):
    return "Amarillo"
  elif (number == 5):
    return "Rojo"
  elif (number == 6):
    return "Verde"
  elif (number == 7):
    return "Naranja"
  else:
    return "sin Color"

# Genera escuelas profesionales
def generate_professional_school(number):
  if (number == 1):
    return "Ingeniería de Software"
  elif (number == 2):
    return "Medicina"
  elif (number == 3):
    return "Derecho"
  elif (number == 4):
    return "Arquitectura"
  elif (number == 5):
    return "Psicología"
  elif (number == 6):
    return "Contabilidad"
  elif (number == 7):
    return "Economía"
  else:
    return "sin escuela"

#Genera registro en csv según formato especificado
def writeAluCSVfile(i):
  f = open("alumno1M.csv", "a")
  #(AP,Nom,edad,sexo,peso,talla,color,prov); 1:M 2=F
  f.write(
      str(i) + "," +
      "Apellidos " + str(i) + " " + generate_name(1, generate_random_int(6, 20)) + "," +
      "Nombres " + str(i) + " " + generate_name(1, generate_random_int(6, 20)) + "," +
      str(generate_random_int(18, 80)) + "," +
      str(generate_random_int(1, 2)) + "," +
      str(generate_random_int(30, 120)) + "," +
      str(generate_random_float(1.1, 2.4)) + "," +
      generate_color(generate_random_int(1, 7)) + "," +
      generate_province(generate_random_int(1, 7)) + "," +
      generate_professional_school(generate_random_int(1, 7)) + "," +
      generate_random_date(2000, 2025) + "\n"
  )
  f.close()

#Genera csv según registros especificado
def write_longdataAlu(number):
  for i in range(1, number + 1):
    writeAluCSVfile(str(i))
  print("written...")

#Ejecuta creación de csv con 1000000 de registros
write_longdataAlu(1000000)

written...


## **2. Crear pipeline**

Crea archivo shell con pipeline que genere resultados especificados sobre data de alumnos.

In [26]:
%%bash
cat > contar.sh << 'EOF'
#!/bin/bash

archivo=$1

# Inicializa variables
varones=0
mujeres=0
declare -A escuelas
total_edad=0
contador_edad=0
ingresos_desde_2021=0

# Funciones contadoras
contar_sexo() {
  if [ "$1" -eq 1 ]; then
    varones=$((varones + 1))
  elif [ "$1" -eq 2 ]; then
    mujeres=$((mujeres + 1))
  fi
}
contar_escuela() {
  escuelas["$1"]=$((escuelas["$1"] + 1))
}
contar_y_sumar_edad() {
  total_edad=$((total_edad + $1))
  contador_edad=$((contador_edad + 1))
}
contar_ingresos_desde_2021() {
  if [[ "$1" > "2021-01-01" ]]; then
    ingresos_desde_2021=$((ingresos_desde_2021 + 1))
  fi
}
calcular_promedio_edad() {
  if [ "$contador_edad" -gt 0 ]; then
    promedio_edad=$((total_edad / contador_edad))
  else
    promedio_edad=0
  fi
}

# Procesa el archivo línea por línea
while IFS=, read -r id apellidos nombres edad sexo peso talla color provincia escuela fecha_ingreso
do
  contar_sexo "$sexo"
  contar_escuela "$escuela"
  contar_y_sumar_edad "$edad"
  contar_ingresos_desde_2021 "$fecha_ingreso"
done < "$archivo"

# Calcula el promedio de edad
calcular_promedio_edad

# Muestra resultados
echo "Ejemplo de pipeline para contar datos de alumnos"
echo "------------------------------------------------"
echo "Cantidad de varones: $varones"
echo "Cantidad de mujeres: $mujeres"
echo "Promedio de edad: $promedio_edad"
echo "Cantidad de personas que ingresaron desde 2021-01-01: $ingresos_desde_2021"
echo "Cantidad de alumnos por cada Escuela Profesional:"
for escuela in "${!escuelas[@]}"
do
  echo "$escuela: ${escuelas[$escuela]}"
done
echo "Fin del pipeline!"

bash: line 66: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')


Agrega permisos de ejecución de archivo shell.

In [27]:
%%bash
chmod +x contar.sh

## **3. Ejecución**

Ejecuta archivo shell (pipeline) sobre data.

In [28]:
%%bash
./contar.sh alumno1M.csv

Process is interrupted.
